In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from typing import Dict, List, Tuple, Optional
from pathlib import Path
import numpy as np
import pandas as pd
from pathlib import Path
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.dataSession import DataSession
from neuropy.core.session.Formats.SessionSpecifications import SessionFolderSpec, SessionFileSpec

# For specific load functions:
from neuropy.core import DataWriter, NeuronType, Neurons, BinnedSpiketrain, Mua, ProbeGroup, Position, Epoch, Signal, Laps, FlattenedSpiketrains, Shank, Probe, ProbeGroup
from neuropy.io import OptitrackIO, PhyIO
from neuropy.utils.mixins.print_helpers import ProgressMessagePrinter, SimplePrintable, OrderedMeta

from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder, DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.utils.result_context import IdentifyingContext


## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/home/halechr/FastData'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/home/halechr/cloud/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"


## Rachel:
active_data_mode_name = 'rachel'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('Rachel')

# basedir: Path = Path(r'W:\Data\Rachel\20230614_Rachel').resolve()

# basedir: Path = Path('/home/halechr/FastData/Rachel/20230614_Rachel/merged_20230614_2crs.GUI').resolve()

basedir: Path = Path('/home/halechr/FastData/Rachel/20230614_Rachel').resolve()
assert basedir.exists()

# filename: str = '20230614_Rachel'
# filename: str = '20230614_Rachel_2'
filename: str = 'merged_20230614_2'

Automatic pdb calling has been turned OFF


AssertionError: 

In [2]:

# RachelDataSessionFormat.initialize_data_directory(basedir)
## Builds the .neurons.npy:
# folder = Path('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/merged_M1_20211123_raw_phy')
# folder = Path(r'W:\Data\Rachel\20230614_Rachel')
folder = basedir.resolve()
phydata = PhyIO(folder)
# /home/halechr/FastData/Rachel/20230614_Rachel/params.py

# phydata.
# neuronIDs = pd.read_csv(r'W:\Data\Rachel\20230614_Rachel\cluster_q.tsv');
phydata.shank_ids
neuronIDs = pd.read_csv(basedir.joinpath('cluster_q.tsv'), sep='\t')
print(f'neuronIDs: {neuronIDs}')


id column does not exist in cluster_info.tsv. Using cluster_id column instead.
neuronIDs:      cluster_id  q
0             0  0
1             3  0
2             5  0
3             6  0
4            12  0
..          ... ..
284        1381  0
285        1382  0
286        1388  0
287        1389  1
288        1390  1

[289 rows x 2 columns]


In [11]:
len(neuronIDs)

289

In [ ]:
# neuronIDs.cluster_id.to_numpy()
neuronIDs.q.to_numpy()

In [ ]:
# [NeuronType.from_qclu_series(a_q) for a_q in neuronIDs.q.to_numpy()]

NeuronType.from_qclu_series(neuronIDs.q.to_numpy())

In [13]:
neurons = Neurons(spiketrains=phydata.spiketrains, t_stop=53602.047, sampling_rate=30000, neuron_ids=neuronIDs.cluster_id.to_numpy(), neuron_type=NeuronType.from_qclu_series(neuronIDs.q.to_numpy()), shank_ids=np.array([0]*len(neuronIDs))) # np.arange(len(neuron_IDs))
				#    neuron_type=neuronIDs)

	
# {1:'pyr1',2:'pyr2',3:'pyr3',4:'int1',5:'int2',6:'int3',7:"mua1",8:'mua2',9:'mua3'}
# neurons = Neurons(spiketrains=phydata.spiketrains, t_stop=2*3600, sampling_rate=30000, neuron_ids = {1:'pyr1',2:'pyr2',3:'pyr3',4:'int1',5:'int2',6:'int3',7:"mua1",8:'mua2',9:'mua3'})

# neuronIDs
neurons

Neurons
 n_neurons: 63
 n_total_spikes: 1166460
 t_start: 0.0
 t_stop: 53602.047

In [14]:
neurons.filename = folder.joinpath(f'{filename}.neurons.npy')
neurons.save()


merged_20230614_2.neurons.npy saved


In [15]:

# Probe Groups file
# TODO: Probe group generation
# shanks = []
# # channel_groups = sess.recinfo.channel_groups
# for i in range(8):
#     shank = Shank.auto_generate(
#         columns=1,
#         contacts_per_column=128,
#         xpitch=90,
#         ypitch=0,
#         y_shift_per_column=[0, 0],
#         channel_id=np.arange(0,128,1)
#         ),
	
# elec_IDs = np.arange(0,128,1)
# shanks = Shank.auto_generate(channels=1, contacts_per_column = 128)
# shanks = pd.read_csv('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/Probe.csv',delimiter=',',usecols=["ShankNumber"])
# shanks = pd.read_csv(folder.joinpath(f'{filename}.Probe.csv'),delimiter=',',usecols=["ShankNumber"])
# prb = Probe(shanks)
# prbgroup = ProbeGroup()
# prbgroup.add_probe(prb)


## Builds the .position.npy:
# opti_folder = Path(r'W:\Data\Rachel\20230614_Rachel')
# opti_folder = basedir.resolve()
# opti_data = OptitrackIO(opti_folder)
# brelative = pd.read_csv(r'W:\Data\Rachel\20230614_Rachel\merged_M1_20211123_raw_behavior_relativetoLFP.csv',header = None)

csv_path = basedir.joinpath(f'20230614_positionData.csv')
# brelative = pd.read_csv(csv_path, header = None)
brelative = pd.read_csv(csv_path)
brelative
# Change column type to timedelta64[ns] for column: 'AbsoluteTime'
brelative = brelative.astype({'AbsoluteTime': 'timedelta64[ns]'})

brelative_seconds = brelative.AbsoluteTime.dt.total_seconds().to_numpy()
start_t = np.min(brelative_seconds)
# start_t = np.min(brelative.AbsoluteTime.to_numpy())
print(f'start_t: {start_t}')

t_relative = brelative_seconds - start_t
t_relative

print(f'brelative.shape: {brelative.shape}')
# d = {'t':brelative[0],'x':opti_data.z,'y':opti_data.x} 
d = {'t':brelative_seconds,'x':brelative.Z.to_numpy(),'y':brelative.X.to_numpy()} 

behaviordf = pd.DataFrame(data=d)
print(f'behaviordf.shape: {behaviordf.shape}')
position = Position(behaviordf)
# position.filename = Path(f'W:\Data\Rachel\20230614_Rachel\{filename}.position.npy')
position.filename = basedir.joinpath(f'{filename}.position.npy')
position.save()

## Builds the .paradigm.npy file from scratch:
# starts = [0, 5*60]
# stops = [5*60-1, 3.8398632e+03]
# labels = ['pre','maze']

paradigm_df = pd.DataFrame(dict(label=['pre','maze','post'],
	start = ['11:15:49.000','12:02:19.095','13:08:37.999'],
	stops = ['11:53:19.384','13:04:54.815','14:53:22.047'],
))
# Change column type to timedelta64[ns] for columns: 'Starts', 'Stops'
paradigm_df = paradigm_df.astype({'start': 'timedelta64[ns]', 'stops': 'timedelta64[ns]'})


## Build and save the paradigm.npy
d = {'start':paradigm_df.start.dt.total_seconds().to_numpy(),
	 'stop':paradigm_df.stops.dt.total_seconds().to_numpy(),
	 'label':paradigm_df.label.to_list()} 
paradigmdf = pd.DataFrame(data=d)
paradigm = Epoch(paradigmdf)
# paradigm.filename = Path('/home/wahlberg/Exp_Data/M1_Nov2021/20211123/merged_M1_20211123_raw/merged_M1_20211123_raw_phy/merged_M1_20211123_raw.paradigm.npy')
paradigm.filename = basedir.joinpath(f'{filename}.paradigm.npy')
paradigm.save()

start_t: 43339.095
brelative.shape: (221455, 4)
behaviordf.shape: (221455, 3)
merged_20230614_2.position.npy saved
merged_20230614_2.paradigm.npy saved


In [ ]:
paradigmdf

In [16]:
# shanks = pd.read_csv(folder.joinpath(f'{filename}.Probe.csv'),delimiter=',',usecols=["ShankNumber"])
# shanks = pd.read_csv(folder.joinpath(f'{filename}.Probe.csv'))
# Change column type to float64 for column: 'ShankNumber'
# shanks = shanks.astype({'ShankNumber': 'float64'})

# elec_IDs = np.arange(0,128,1)
# shanks = Shank.auto_generate(channels=1, contacts_per_column = 128)

shanks = Shank.auto_generate(columns=1, contacts_per_column = 128, xpitch=0, ypitch=0, channel_id = np.arange(127,-1,-1))

# shanks_list = [Shank.auto_generate(columns=1, contacts_per_column = 32, xpitch=0, ypitch=0, channel_id = np.arange(31,-1,-1)) for i in np.arange(4)]
shanks


In [17]:
prb = Probe(shanks)
# prb = Probe(list(np.arange(128)))
prbgroup = ProbeGroup()
prbgroup.add_probe(prb)
prbgroup.filename = basedir.joinpath(f'{filename}.probegroup.npy')
prbgroup.save(prbgroup.filename)

merged_20230614_2.probegroup.npy saved


/home/halechr/repos/NeuroPy/neuropy/core/probe.py:149: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._data = self._data.append(shank_df)
/home/halechr/repos/NeuroPy/neuropy/core/probe.py:341: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._data = self._data.append(probe_df)
/home/halechr/repos/NeuroPy/neuropy/core/probe.py:341: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  self._data = self._data.append(probe_df)


In [18]:

_test_session = RachelDataSessionFormat.build_session(basedir)
_test_session

/home/halechr/repos/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:123: UserWarning: WARNING: Optional File: /home/halechr/FastData/Rachel/20230614_Rachel/merged_20230614_2.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


DataSession(config: rachel_20230614_Rachel_sess): Not yet configured.

In [ ]:
_test_session.neurons

In [22]:
_test_session, loaded_file_record_list = RachelDataSessionFormat.load_session(_test_session)
_test_session


Failure loading .flattened.spikes.npy. Must recompute.

Computing flattened_spike_identities results : build_spike_dataframe(...)... 

/home/halechr/repos/NeuroPy/neuropy/core/session/Formats/SessionSpecifications.py:123: UserWarning: WARNING: Optional File: /home/halechr/FastData/Rachel/20230614_Rachel/merged_20230614_2.dat does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


done.
Computing flattened_spike_types results : build_spike_dataframe(...)... done.
Computing flattened_spike_shank_identities results : build_spike_dataframe(...)... done.
Computing flattened_spike_linear_unit_spike_idx results : build_spike_dataframe(...)... done.
Computing flattened_spike_times results : build_spike_dataframe(...)... done.
Sorting flattened_sort_indicies results : build_spike_dataframe(...)... done.
Building final dataframe (containing 1166460 spikes) spikes_df results : build_spike_dataframe(...)... done.
spikes_df.columns: Index(['flat_spike_idx', 't_seconds', 'aclu', 'unit_id', 'shank',
       'intra_unit_spike_idx', 'neuron_type'],
      dtype='object')
	 Done!
	 force_recompute is True! Forcing recomputation of .interpolated_spike_positions.npy

Computing interpolate_spike_positions columns results : spikes_df... done.


In [ ]:
# ValueError: shape mismatch: value array of shape (2,221425) could not be broadcast to indexing result of shape (221425,)